In [2]:
import praw
import pandas as pd
import os
import time
from datetime import datetime, timezone
from tqdm import tqdm

# --- Reddit API Credentials ---
REDDIT_CLIENT_ID = "lljpdrQyqGDtoFKbuk-BeA"
REDDIT_CLIENT_SECRET = "WmDEAFCMvFhjxsjxaFz7EZtr-d9opw"
REDDIT_USER_AGENT = "MentalHealthScraper:v1.0 (by u/aipromi)"

# --- Data Directory ---
DATA_DIR = "Data"
os.makedirs(DATA_DIR, exist_ok=True)

# --- Subreddits to scrape ---
SUBREDDITS = [
    'suicidewatch', 'depression', 'ptsd', 'anxiety', 'socialanxiety', 'healthanxiety', 'bipolarreddit',
    'bpd', 'schizophrenia', 'paranoia', 'EDAnonymous', 'alcoholism', 'addiction', 'adhd', 'mindfulness',
    'psychosis', 'mentalillness', 'mentalhealth', 'meditation', 'personalfinance', 'jokes', 'teaching',
    'relationships', 'legaladvice', 'fitness', 'parenting', 'COVID19_support', 'lonely', 'autism', 'divorce',
    'guns', 'economy', 'ForeverAlone', 'conspiracy', 'MakeNewFriendsHere', 'AskReddit', 'offmychest', 'LifeProTips',
    'TrueOffMyChest', 'unpopularopinion', 'CasualConversation', 'Advice', 'selfimprovement', 'nosleep', 'todayilearned',
    'AskMen', 'AskWomen', 'AntiMLM', 'changemyview', 'confession', 'decidingtobebetter', 'mentalhealthUK', 'support',
    'stopdrinking', 'getdisciplined', 'tifu', 'careerguidance', 'psychology', 'Wellthatsucks', 'selfcare', 'Stress',
    'socialskills', 'Therapy', 'friendship', 'caregivers', 'Loneliness', 'sad', 'BPDlovedones', 'aspergers', 'autismparents'
]

# --- Initialize Reddit instance ---
reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent=REDDIT_USER_AGENT
)

def clean_text(txt):
    if isinstance(txt, str):
        return txt.replace('\n', ' ').replace('\r', ' ').strip()
    return ''

def get_scraped_ids(filepath):
    """Return set of post_ids already saved in CSV, if file exists."""
    if os.path.exists(filepath):
        try:
            df = pd.read_csv(filepath, usecols=["post_id"])
            return set(df["post_id"].astype(str))
        except Exception:
            return set()
    return set()

def scrape_subreddit(subreddit, post_limit=None, comment_limit=50):
    posts_csv = os.path.join(DATA_DIR, f"{subreddit}_posts.csv")
    comments_csv = os.path.join(DATA_DIR, f"{subreddit}_comments.csv")
    scraped_ids = get_scraped_ids(posts_csv)
    posts = []
    comments = []
    new_posts = 0

    try:
        subreddit_obj = reddit.subreddit(subreddit)
        for submission in tqdm(subreddit_obj.new(limit=post_limit), desc=f"r/{subreddit} posts"):
            if submission.id in scraped_ids:
                continue
            post_id = submission.id
            # FIXED: Use timezone-aware UTC datetime
            post_data = {
                'post_id': post_id,
                'content': clean_text(submission.title) + ' ' + clean_text(submission.selftext),
                'author': str(submission.author) if submission.author else '',
                'date': datetime.fromtimestamp(submission.created_utc, timezone.utc).isoformat() if hasattr(submission, 'created_utc') else '',
                'category': subreddit,
                'num_comments': submission.num_comments,
                'score': submission.score,
                'upvote_ratio': submission.upvote_ratio,
                'url': submission.url
            }
            posts.append(post_data)
            new_posts += 1

            # Fetch comments (up to comment_limit)
            try:
                submission.comments.replace_more(limit=0)
                comment_list = submission.comments.list()
                comment_list = sorted(comment_list, key=lambda x: getattr(x, 'created_utc', 0))
                for idx, c in enumerate(comment_list[:comment_limit]):
                    comments.append({
                        'comment_id': getattr(c, 'id', ''),
                        'post_id': post_id,
                        'content': clean_text(getattr(c, 'body', '')),
                        'author': str(getattr(c, 'author', '')) if getattr(c, 'author', None) else '',
                        'date': datetime.fromtimestamp(getattr(c, 'created_utc', 0), timezone.utc).isoformat() if hasattr(c, 'created_utc') else '',
                        'order': idx + 1,
                        'score': getattr(c, 'score', ''),
                        'parent_id': getattr(c, 'parent_id', ''),
                        'link_id': getattr(c, 'link_id', ''),
                        'permalink': getattr(c, 'permalink', '')
                    })
            except Exception as ce:
                print(f"Error fetching comments for post {post_id} in r/{subreddit}: {ce}")

            # Save progress every 100 posts
            if new_posts % 100 == 0:
                if posts:
                    pd.DataFrame(posts).to_csv(posts_csv, mode='a', header=not os.path.exists(posts_csv), index=False)
                    posts = []
                if comments:
                    pd.DataFrame(comments).to_csv(comments_csv, mode='a', header=not os.path.exists(comments_csv), index=False)
                    comments = []
                time.sleep(2)  # brief pause

    except Exception as e:
        print(f"Error scraping r/{subreddit}: {e}")

    # Save any remaining data
    if posts:
        pd.DataFrame(posts).to_csv(posts_csv, mode='a', header=not os.path.exists(posts_csv), index=False)
    if comments:
        pd.DataFrame(comments).to_csv(comments_csv, mode='a', header=not os.path.exists(comments_csv), index=False)
    print(f"Saved {new_posts} new posts for r/{subreddit}.")
    time.sleep(30)  # pause between subreddits

# Main loop
for sub in tqdm(SUBREDDITS, desc="Subreddits"):
    scrape_subreddit(sub, post_limit=None, comment_limit=50)  # None = as many as Reddit allows

Subreddits:   0%|          | 0/70 [00:00<?, ?it/s]
r/suicidewatch posts: 0it [00:00, ?it/s]
r/suicidewatch posts: 1it [00:03,  3.71s/it]
r/suicidewatch posts: 2it [00:04,  1.77s/it]
r/suicidewatch posts: 3it [00:04,  1.24s/it]
r/suicidewatch posts: 4it [00:06,  1.31s/it]
r/suicidewatch posts: 5it [00:06,  1.03s/it]
r/suicidewatch posts: 6it [00:07,  1.22it/s]
r/suicidewatch posts: 7it [00:07,  1.46it/s]
r/suicidewatch posts: 8it [00:07,  1.66it/s]
r/suicidewatch posts: 9it [00:08,  1.86it/s]
r/suicidewatch posts: 10it [00:08,  2.01it/s]
r/suicidewatch posts: 11it [00:09,  1.99it/s]
r/suicidewatch posts: 12it [00:09,  2.11it/s]
r/suicidewatch posts: 13it [00:10,  2.20it/s]
r/suicidewatch posts: 14it [00:10,  2.28it/s]
r/suicidewatch posts: 15it [00:10,  2.32it/s]
r/suicidewatch posts: 16it [00:11,  2.35it/s]
r/suicidewatch posts: 17it [00:11,  2.38it/s]
r/suicidewatch posts: 18it [00:12,  2.40it/s]
r/suicidewatch posts: 19it [00:12,  2.41it/s]
r/suicidewatch posts: 20it [00:12,  2.42it/

Saved 223 new posts for r/suicidewatch.


Subreddits:   1%|▏         | 1/70 [02:35<2:59:21, 155.96s/it]
r/depression posts: 0it [00:00, ?it/s]
r/depression posts: 1it [00:02,  2.17s/it]
r/depression posts: 2it [00:02,  1.14s/it]
r/depression posts: 3it [00:02,  1.30it/s]
r/depression posts: 4it [00:03,  1.52it/s]
r/depression posts: 5it [00:03,  1.76it/s]
r/depression posts: 6it [00:04,  1.93it/s]
r/depression posts: 7it [00:04,  2.09it/s]
r/depression posts: 8it [00:05,  2.17it/s]
r/depression posts: 9it [00:05,  2.26it/s]
r/depression posts: 10it [00:05,  2.34it/s]
r/depression posts: 11it [00:06,  2.45it/s]
r/depression posts: 12it [00:06,  2.54it/s]
r/depression posts: 13it [00:06,  2.51it/s]
r/depression posts: 14it [00:07,  2.48it/s]
r/depression posts: 15it [00:07,  2.47it/s]
r/depression posts: 16it [00:08,  2.47it/s]
r/depression posts: 17it [00:08,  2.45it/s]
r/depression posts: 18it [00:09,  2.47it/s]
r/depression posts: 19it [00:09,  2.44it/s]
r/depression posts: 20it [00:09,  2.43it/s]
r/depression posts: 21it [00

Saved 144 new posts for r/depression.


Subreddits:   3%|▎         | 2/70 [04:25<2:26:02, 128.86s/it]
r/ptsd posts: 0it [00:00, ?it/s]
r/ptsd posts: 1it [00:02,  2.44s/it]
r/ptsd posts: 2it [00:02,  1.25s/it]
r/ptsd posts: 3it [00:03,  1.15it/s]
r/ptsd posts: 4it [00:03,  1.46it/s]
r/ptsd posts: 5it [00:04,  1.74it/s]
r/ptsd posts: 6it [00:04,  1.90it/s]
r/ptsd posts: 7it [00:04,  2.04it/s]
r/ptsd posts: 8it [00:05,  2.22it/s]
r/ptsd posts: 9it [00:05,  2.21it/s]
r/ptsd posts: 10it [00:06,  2.35it/s]
r/ptsd posts: 11it [00:06,  1.77it/s]
r/ptsd posts: 12it [00:07,  1.84it/s]
r/ptsd posts: 13it [00:07,  2.00it/s]
r/ptsd posts: 14it [00:08,  2.11it/s]
r/ptsd posts: 15it [00:08,  2.06it/s]
r/ptsd posts: 16it [00:09,  1.47it/s]
r/ptsd posts: 17it [00:10,  1.68it/s]
r/ptsd posts: 18it [00:10,  1.74it/s]
r/ptsd posts: 19it [00:11,  1.80it/s]
r/ptsd posts: 20it [00:11,  1.97it/s]
r/ptsd posts: 21it [00:12,  2.08it/s]
r/ptsd posts: 22it [00:12,  2.04it/s]
r/ptsd posts: 101it [00:14, 25.18it/s]
r/ptsd posts: 201it [00:16, 37.33it/s]


...

Subreddits:  97%|█████████▋| 68/70 [1:29:49<01:41, 50.93s/it]
r/aspergers posts: 0it [00:00, ?it/s]
r/aspergers posts: 1it [00:02,  2.68s/it]
r/aspergers posts: 2it [00:03,  1.36s/it]
r/aspergers posts: 3it [00:03,  1.08it/s]
r/aspergers posts: 4it [00:03,  1.38it/s]
r/aspergers posts: 5it [00:04,  1.64it/s]
r/aspergers posts: 6it [00:04,  1.74it/s]
r/aspergers posts: 7it [00:05,  1.81it/s]
r/aspergers posts: 8it [00:05,  1.85it/s]
r/aspergers posts: 9it [00:06,  2.00it/s]
r/aspergers posts: 10it [00:06,  1.99it/s]
r/aspergers posts: 11it [00:07,  1.97it/s]
r/aspergers posts: 12it [00:07,  2.10it/s]
r/aspergers posts: 47it [00:08, 24.82it/s]
r/aspergers posts: 101it [00:10, 27.29it/s]
r/aspergers posts: 201it [00:11, 40.04it/s]
r/aspergers posts: 301it [00:13, 44.67it/s]
r/aspergers posts: 401it [00:15, 45.35it/s]
r/aspergers posts: 501it [00:17, 48.33it/s]
r/aspergers posts: 601it [00:19, 52.60it/s]
r/aspergers posts: 701it [00:20, 55.45it/s]
r/aspergers posts: 801it [00:22, 55.13it/s

Saved 13 new posts for r/aspergers.


Subreddits:  99%|█████████▊| 69/70 [1:30:44<00:52, 52.05s/it]
r/autismparents posts: 0it [00:00, ?it/s]
r/autismparents posts: 44it [00:01, 29.34it/s][A


Saved 0 new posts for r/autismparents.


Subreddits: 100%|██████████| 70/70 [1:31:16<00:00, 78.23s/it]
